# Practica 4
## Captcha Destroyer hecho por: Python Haters
#### - Hugo Vivanco Fernandez
#### - Jaime Isar Muñoz
#### - Daniel Lafuente Bazo
#### - Óscar Fabián Pineda Germán

## Parte 0: Tratamiento de imagenes
Trabajamos con .png y tenemos que ser capaces de procesarlos, para esto usaremos cv2 (OpenCV)

In [9]:

import tensorflow as tf
from tensorflow.keras import layers, models
from keras.models import Model
from tensorflow.keras.models import load_model
from PIL import Image
import random
import numpy as np
import string
import cv2
import os


symbols = string.ascii_lowercase + string.digits    # Todos los digitos que contienen los CAPTCHAs
directorio_input = 'samples/'                       # Directorio de CAPTCHAs sin filtrar
directorio_output_filtrado = 'samples_transform/'            # Directorio de CAPTCHAs como imagen binaria
CAPTCHA_LENGHT = 5                                  # Numero de caracteres por CAPTCHA                                  
formato_imagen = (50, 200, 1)                       # Contador de imagenes procesadas en total


def convertir_imagen(ruta_input, ruta_output, dimension=(50, 200)):

    # Abrir imagen
    imagen = Image.open(ruta_input)
    
    # Cambiar tamaño
    imagen = imagen.resize(dimension)
    
    # Convertir a blanco y negro
    imagen_bn = imagen.convert('L')  # 'L' es modo de 8 bits en escala de grises
    
    # Guardar la nueva imagen
    imagen_bn.save(ruta_output)

def imagen_a_matriz(ruta_imagen, umbral=128):

    imagen = Image.open(ruta_imagen)    # Abrir imagen
    
    imagen_bn = imagen.point(lambda p: 255 if p > umbral else 0)    # Convertir a imagen binaria
    
    imagen_array = np.array(imagen_bn)  # Tranformar a numpy array

    matriz = np.where(imagen_array == 0, 1, 0)  # Invertir pixeles

    matriz = np.expand_dims(matriz, axis=-1)  # Convertir a: (200, 50, 1)

    return matriz

def codificar_solucion(nombre_imagen : str):

    symbols_size = len(symbols)
    one_hot = np.zeros((CAPTCHA_LENGHT, symbols_size))
    index = 0
    for char in nombre_imagen:
        index_in_symbols = symbols.find(char)
        one_hot[index, index_in_symbols] = 1    # hot_one codification
        index += 1
    
    return one_hot

def obtener_datos_de_imagenes():

    datos = []          # (n*200*50*1): Num datos * 200 * 50 * 1
    soluciones = []     # Solucion para cada imagen
    contador = 0        # Contador de imágenes procesadas

    for archivo in os.listdir(directorio_input):
        archivo_nuevo = directorio_output_filtrado + archivo  # Ruta de la imagen
        nombre_imagen = archivo[:-4]  # Quitar el .png para obtener la solución (texto del captcha)

        if os.path.isfile(archivo_nuevo):
            datos.append(imagen_a_matriz(archivo_nuevo))    # Añade matriz de imagen
            soluciones.append(nombre_imagen)                # Añade la solucion de esta imagen 
            contador += 1

    return datos, soluciones, contador

def one_hot_encode(soluciones):

    n = len(soluciones)
    y = np.zeros( (CAPTCHA_LENGHT, n, len(symbols)) )  # (5, n, 36) : 5 letras, n imágenes, 36 posibles caracteres

    for i, texto in enumerate(soluciones):
        for j, letra in enumerate(texto):
            index = symbols.find(letra)
            if index != -1:
                y[j, i, index] = 1

    return y

def createmodel():

    # Bloque de entrada
    lista_neuronas_input = layers.Input(shape=formato_imagen)

    # Primer bloque
    convolutional_layer1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(lista_neuronas_input)
    max_pooling_1 = layers.MaxPooling2D(pool_size=(2, 2))(convolutional_layer1)

    # Segundo bloque
    convolutional_layer2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(max_pooling_1)
    max_pooling_2 = layers.MaxPooling2D(pool_size=(2, 2))(convolutional_layer2)

    # Tercer bloque
    convolutional_layer3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(max_pooling_2)
    batch_normalitation = layers.BatchNormalization()(convolutional_layer3) # Mejora la estabilidad(N) -> Media=0 y desviacion estandar=1
    max_pooling_3 = layers.MaxPooling2D(pool_size=(2, 2))(batch_normalitation)
    flatten_layer = layers.Flatten()(max_pooling_3)

    lista_neuronas_output = []
    for _ in range(5):
        input_layer = layers.Dense(64, activation='relu')(flatten_layer)
        outupt_layer = layers.Dense(len(symbols), activation='sigmoid')(input_layer)
        lista_neuronas_output.append( layers.Dense(36, activation='softmax')(outupt_layer) )


    modelo = models.Model(inputs=lista_neuronas_input, outputs=lista_neuronas_output)
    modelo.compile(loss=['categorical_crossentropy'], optimizer='adam', metrics=['accuracy'] * CAPTCHA_LENGHT)  # Nuestra red tiene 5 salidas, una por letra del captcha
    return modelo

def predict(filepath):
    
    img = imagen_a_matriz(filepath)

    res = np.array( modelo.predict(img) )

    #added this bcoz x_train 970*50*200*1
    #returns array of size 1*5*36 
    result = np.reshape(res, (5, 36)) #reshape the array
    k_ind = []
    for i in result:
        k_ind.append(np.argmax(i)) #adds the index of the char found in captcha

    capt = '' #string to store predicted captcha
    for k in k_ind:
        capt += symbols[k] #finds the char corresponding to the index

    return capt 

def test(modelo, j=20):

    # Listar todas las imágenes en el directorio
    imagenes = [f for f in os.listdir('samples_transform') if f.endswith('.png')]

    # Elegir j imágenes aleatorias
    seleccionadas = random.sample(imagenes, j)

    for nombre_imagen in seleccionadas:

        ruta = os.path.join('samples_transform', nombre_imagen)
        img = imagen_a_matriz(ruta)
        img = np.expand_dims(img, axis=0)  # Añade dimensión batch (1, 50, 200, 1)

        # Pasar la imagen por la red
        predicciones = modelo.predict(img)

        # Decodificar predicciones
        resultado = ''
        for salida in predicciones:
            idx = np.argmax(salida)
            resultado += symbols[idx]

        print(f'Imagen: {nombre_imagen[:-4]} ➔ Predicción: {resultado}')


modelo = ''
if os.path.exists('modelo.h5'):
    modelo = load_model('modelo.h5')
else:
    X_data, Y_data, numImagenes = obtener_datos_de_imagenes()
    Y_data = one_hot_encode(Y_data)

    # Crear modelo
    modelo = createmodel()
    modelo.summary()    # Mostrar caracteristicas

    # División de datos
    split_index = int(numImagenes * 0.8)    # Obtener numero de datos usados para entrenar
    X_train, X_test = X_data[:split_index], X_data[split_index:]    # Datos usados para entrenar y para testear
    Y_train, Y_test = Y_data[:split_index], Y_data[split_index:]    # Soluciones de los datos usados para entrenar y para testear

    # Entrenar modelo
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    hist = modelo.fit(
        X_train,
        [Y_train[0], Y_train[1], Y_train[2], Y_train[3], Y_train[4]],
        batch_size=32,
        epochs=60,
        validation_split=0.2
    )
    modelo.save('modelo.h5')

## Parte 1: Reconocer un dígito o letra deformado

Para empezar el proyecto decidimos ir paso a paso. Primero identificaremos numeros y letras de forma individual. (logic CNN)

In [11]:
test(modelo=modelo)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Imagen: e8e5e ➔ Predicción: ensym
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Imagen: gfbx6 ➔ Predicción: gnvyx
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Imagen: gwnm6 ➔ Predicción: gnxyx
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Imagen: 5f3gf ➔ Predicción: 5nvxx
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Imagen: nbcgb ➔ Predicción: envyw
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Imagen: y53c2 ➔ Predicción: cnsxm
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Imagen: pg4bf ➔ Predicción: d9s1x
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Imagen: 5pm6b ➔ Predicción: 59vdx
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Imagen: 5n3w4 ➔ Predicción: 59vyy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Imagen: men4f ➔ Predicción: g9xyx
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Imagen: fp382 ➔ Predicción: f9vyx
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Imagen: d3c8y ➔ Predicción: dns1m
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Imagen: 4n3mn ➔ Predicción: 4nxyx
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Imagen: 7fde7 ➔ Predicción